# TP1 - Introduction aux réseaux de neurones

---

- <span style="color:orange">**Conçu par:**</span> Prof. Hamza Khalfi

---

## Partie 1 - Formalisation mathématique
Pour appliquer un réseau de neurones à un problème de machine learning (en apprentissage supervisé),
on a besoin de 4 choses :

— Un **jeu de données** annoté et un problème associé ;

— Une architecture de réseau (à adapter aux données) ;

— Une fonction de coût que l’on cherche à minimiser (à adapter au problème) ;

— Un algorithme d’apprentissage pour résoudre le problème d’optimisation consistant à minimiser
cette fonction de coût.

On s'intéresse ici à un problème de classification en apprentissage supervisé. On a donc un jeu de données constitué d'un ensemble de $N$ couples $ (x^{(i)}, y^{(i)}) $, où $ i \in \{1, ..., N\} $, $ x^{(i)} \in \mathbb{R}^{n_x} $ est un vecteur de _features_ à partir duquel on veut prédire la vérité terrain $ y^{(i)} \in \{0, 1\}^{n_y} $ vérifiant $ \|y^{(i)}\| = 1 $ (_one hot encoding_). 

Ce jeu de données est généralement découpé en plusieurs ensembles : **train**, **test** et parfois **val**.

### 1.1 - Jeu de données
#### Q1 - A quoi servent les ensembles d’apprentissage, de validation et de test ?

Étant donnée une fonction _loss_, nous pouvons essayer de trouver la fonction $h$ qui minimise cette _loss_ :
$$ h = \text{argmin}_{h \in H} L(h) $$

_Comment effectuer cette minimisation efficacement ?_

Si vous trouvez une fonction $ h(\cdot) $ avec une faible perte sur vos données $ D $, comment savez-vous si elle classera correctement les exemples qui ne sont pas dans $ D $ ?

Exemple:
$$ h(x) = \begin{cases} 
y_i & \text{si } \exists (x_i,y_i) \in D \text{ t.q. } x = x_i \\
0 & \text{sinon} 
\end{cases} $$

Pour cette $ h(\cdot) $, nous obtenons une erreur de $ 0\% $ sur les données d'entraînement $ D $, mais elle est très inefficace avec des échantillons qui ne sont pas dans $ D $, c'est-à-dire qu'il y a un problème de surapprentissage avec cette fonction.

C'est comme un étudiant qui mémorise toutes les réponses des exercices sans vraiment comprendre les concepts. Lorsqu'il est confronté à de nouvelles questions à l'examen, il échoue car il ne peut pas généraliser ses connaissances.

_Comment diviser l'ensemble Train / Test:_

Pour résoudre le problème du surapprentissage, nous divisons généralement $ D $ en trois sous-ensembles : $ D_{TR} $ comme données d'entraînement, $ D_{VA} $ comme données de validation et $ D_{TE} $ comme données de test. Généralement, ils sont divisés dans une proportion de 80\%, 10\% et 10\%. Ensuite, nous choisissons $ h(\cdot) $ en fonction de $ D_{TR} $ et évaluons $ h(\cdot) $ sur $ D_{TE} $.

_Pourquoi avons-nous besoin de $ D_{VA} $ ?_

$ D_{VA} $ est utilisé pour vérifier si le $ h(\cdot) $ obtenu à partir de $ D_{TR} $ souffre du problème de surapprentissage. $ h(\cdot) $ devra être validé sur $ D_{VA} $. Si la perte est trop grande, $ h(\cdot) $ sera révisé en fonction de $ D_{TR} $ et validé à nouveau sur $ D_{VA} $. Ce processus ira et viendra jusqu'à obtenir une faible perte sur $ D_{VA} $. Il y a un compromis entre les tailles de $ D_{TR} $ et $ D_{VA} $ : les résultats de l'entraînement seront meilleurs pour un $ D_{TR} $ plus grand, mais la validation sera plus fiable (moins bruitée) si $ D_{VA} $ est plus grand.

---

#### Q2 - Quelle est l'influence du nombre $N$ d'exemples?

   - En rapport avec [la loi des grands nombres](https://fr.wikipedia.org/wiki/Loi_des_grands_nombres), un $ N $ plus grand signifie généralement que l'ensemble de données est plus représentatif de la population globale (surtout si les données de base sont bruitées). Cela augmente les chances que le modèle généralise bien aux nouvelles données. Si $ D $ représente la distribution réelle (sous-jacente) des données, alors avec un $ N $ suffisamment grand, la distribution de l'échantillon $ D_{TR} $ s'approche de $ D $.

     $$ \lim_{{N \to \infty}} D_{TR} = D $$

   - Si $ N \ll P $, le nombre des paramètres du modèle, il est possible que le modèle surapprenne (_overfit_) les données. Dans ce cas, $ h(\cdot) $ obtenu à partir de $ D_{TR} $ pourrait avoir une faible perte sur l'ensemble d'entraînement, mais une perte élevée sur $ D_{TE} $ ou $ D_{VA} $.

Plus de détails (comment l'ajout ou la suppression d'un seul (ou quelques) exemple(s) peut affecter le NN):

C'est difficile de donner une réponse rigoureuse à cette question, pour le moment, ce sujet est en cours d'exploration dans le cadre de la théorie de l'apprentissage statistique:

- Les [fonctions d'influence](https://arxiv.org/pdf/1703.04730.pdf) proposées par _Pang Wei Koh_ et _Percy Liang_ qui traite le sujet de comment ces fonctions-là peuvent aider à comprendre le comportement du modèle, à déboguer, à détecter les erreurs dans les jeux de données et à créer des attaques sur l'ensemble d'entraînement.

- Un sujet très intéressant qui est similaire en style au précédent est proposé sous forme de compétition par _Google_ qui s'appelle le [Machine Unlearning](https://www.kaggle.com/competitions/neurips-2023-machine-unlearning) dans le cadre de NeurIPS 2023.

#### Q2i - Comment faire si un bon nombre d'exemples contient des valeurs vides?

Une manière de le faire: 
- On peut changer la fonction coût $J = \sum_i L(y_i, \hat{y_i})$ vers $J = \sum_i w_i L(y_i, \hat{y_i})$.
- Après, on ajuste la pondération de pertinence (selon l'objectif). Par exemple, utilisez `1.0` pour les exemples complets et ``0,1`` pour les incomplets.
- Selon le cadre NN, il pourrait déjà proposer une pondération par exemple, comme le montre [Example-Weighted Neural Network Training](https://reference.wolframcloud.com/language/tutorial/NeuralNetworksExampleWeighting.html). Même si ce n'est pas le cas, grâce à l'_auto-differentiation_, il suffit généralement d'intégrer la logique _forward_ dans la fonction de coût pour chaque minibatch. La pondération est alors correctement appliquée aux gradients. Si on doit calculer les gradients, on multiplie simplement le gradient initial de chaque exemple par son poids $w_i$.
- Après modification, il faut considérer comment configurer et interpréter l'ensemble de test. En production, il faut éviter de pondérer les métriques pour les entrées manquantes.

#### Q2ii - Pourquoi est-ce que les modèles du _deep learning_ demandent (généralement) un nombre $N$ d'exemples beaucoup plus grands que les modèles du _machine learning traditionnel_?

- En principe, Les NN sont très puissants en termes de flexibilité ([Théorème d'appoximation universelle](https://fr.wikipedia.org/wiki/Th%C3%A9or%C3%A8me_d%27approximation_universelle)). Ils sont conçus pour modéliser et optimiser des problèmes beaucoup plus compliqués que ce que le _traditional ML_ n'en est capable.

- En pratique, $P$ le nombre de paramètres est très grand. Par exemple, disons que vous avez 100 neurones dans la première couche, 50 neurones dans la deuxième couche et 1 neurone dans la couche de sortie. Le nombre de paramètres que vous devez optimiser est 10050 + 501 = 5050 (en omettant les paramètres de biais). Pour un modèle de régression linéaire avec 100 caractéristiques, vous n'avez besoin de trouver que 100 coefficients (en omettant à nouveau le biais/interception). Pour la reconnaissance d'images, les réseaux utilisés sont généralement beaucoup plus grands. De plus, il y a aussi d'autres hyperparamètres que vous devez régler dans chaque couche (par exemple, régularisation, _dropout_, fonction d'activation).

#### Q2iii - Est-ce que plus de données est vraiment toujours mieux en apprentissage automatique?

- Pour des données bruitées, plus de données sont préférables, principalement en raison de la loi des grands nombres. En apprentissage statistique, il existe des bornes sur la minimisation du risque empirique. C'est-à-dire, si les données sont _iid_ (indépendantes et identiquement distribuées)., il y a une borne supérieure au risque empirique. (référence:  [Explaining Domain Generalization Via Model Complexity](https://openreview.net/pdf?id=o6dG7nVYDS). )

Supposons que nous ne nous entraînons que sur des données jusqu'à $ 1 ≪ n ≪ N $. Existe-t-il un théorème général montrant que s'entraîner sur $ (x_i, y_i)_{i=1}^n $ conduira à un ajustement moins bon, c'est-à-dire que nous ne serons pas capables de trouver les mêmes paramètres idéaux qu'avec $ (x_i, y_i)_{i=1}^N $ ou est-ce que seul le taux de convergence est plus lent?

- Il ne s'agit pas uniquement de la quantité, mais également si les données sont pertinentes, non biaisées, à quel niveau sont-elles bruitées, de la manière dont elles ont été échantillonnées, etc. Autrement dit, il s'agit de la qualité des données. Si vous disposez des données de bonne qualité, en avoir plus est bénéfique. Cependant, après un certain point, vous atteignez un rendement décroissant où l'ajout de nouvelles données ne présente tout simplement plus d'intérêt. (Pour plus de détails: [Statistical paradises and paradoxes in Big Data](https://www.youtube.com/watch?v=8YLdIDOMEZs&ab_channel=RoyalStatSoc))

**Important**: Il faut noter que ces réponses ne couvrent pas l'ensemble de la thématique.

---

### 1.2 - Architecture du réseau (phase forward)

Un réseau de neurones $f$ est une succession de transformations mathématiques permettant de passer de
l’espace des _features_ à l’espace des prédictions :

$$
f : \mathbb{R}^{n_x} \rightarrow \mathbb{R}^{n_y}
$$

$$
x \mapsto \hat{y}
$$

Le calcul de la sortie $\hat{y}$ à partir de l’entrée $x$ s’appelle la _phase forward_ du réseau.
Un réseau de neurones très simple peut par exemple consister en une simple transformation linéaire, on
aura alors $\hat{y} = f(x) = Wx$ avec $W$ une matrice de taille $n_y \times n_x$. Cette transformation est la couche de base des réseaux de neurones classiques. On utilise d’ailleurs souvent une transformation affine $f(x) = Wx + b$.

Généralement, on fait suivre chaque transformation linéaire d’une fonction d’activation qui est
une fonction mathématique non-linéaire dont le rôle est double. Tout d’abord, elle permet de rendre la
combinaison des transformations non-linéaire, permettant ainsi d’appliquer plusieurs
transformations de suite. Ensuite, elle permet de choisir un intervalle d’arrivée différent de ${R}^{n_y}$, par exemple, $tanh$ permet d’avoir un résultat dans l’intervalle ${[−1, 1]}^{n_y}$ .

Les fonctions d’activations les plus fréquentes sont :

— $ReLU$ (_Rectified Linear Unit_) qui ramène à $0$ les valeurs négatives: $ReLU(x) = max(0, x)$ ;

— $tanh$ permettant de se ramener dans l’intervalle $[−1, 1]$ ;

— $\sigma$ (_sigmoid_) permettant de se ramener dans l’intervalle $[0, 1]$:  $\sigma(x) = 1/(1 + exp(x))$ ;

— _softmax_ permettant de se ramener dans l’intervalle $[0, 1]$ et tel que $\sum_i softmax(x)_i = 1$, permettant
ainsi d’avoir un vecteur assimilable à une distribution de probabilité: $softmax(x)_i = exp(x_i)/ \sum_j exp(x_j)$.

Comme indiqué précédemment, on aura généralement dans un réseau de neurones une succession de
couches linéaires chacune suivie d’une fonction d’activation. La [figure 1](../resources/nn_figure.PNG) présente un réseau assez
simple à une couche cachée (les fonctions d’activation ne sont pas représentées).

Dans le cadre de ce TP, on va s’intéresser à l’architecture suivante :

— Une couche “cachée” passant de l’entrée de taille $n_x$ à un vecteur $h$ de taille $n_h$, constituée d'une transformation affine utilisant une matrice de poids $W_h$ de taille $n_h \times n_x$ et un vecteur de biais $b_h$ de taille $n_h$ ; _On notera $ \tilde{h} $ le vecteur résultant de cette transformation._
et la fonction d'activation $tanh$; _On notera $h$ le vecteur résultant de cette transformation._

— Une couche de sortie passant de la représentation cachée de taille $n_h$ à un vecteur de sortie $\hat{y}$ de
taille $n_y$, constituée d'une transformation affine utilisant une matrice de poids $W_y$ de taille $n_y \times n_h$ et un vecteur de biais $b_y$ de taille $n_y$ ; _On notera $ \tilde{y} $ le vecteur résultant de cette transformation_ et la fonction d'activation $softmax$; _On notera $ \hat{y} $ le vecteur résultant de cette transformation_

#### Q3 - Pourquoi est-il important d’ajouter des fonctions d’activation entre des transformations linéaires ?

- En raison d'expressivité (principalement la non-linéarité): [Théorème d'approximation universelle](https://mcneela.github.io/machine_learning/2017/03/21/Universal-Approximation-Theorem.html)

- En raison d'interprétabilité: Les fonctions d'activation peuvent aider à comprendre le rôle et l'activité de chaque neurone au sein du réseau.

#### Q4 - En pratique, comment ces tailles $n_x$, $n_h$, $n_y$ sont-elles choisies ?

Dans la [_figure 1_](../resources/nn_figure.PNG), $n_x = 2$, $n_h = 4$, $n_y = 2$

**Couche d'entrée:**

Ce paramètre est entièrement et uniquement déterminé une fois que vous connaissez la forme de vos données d'entraînement. Plus précisément, le nombre de neurones composant cette couche est égal au nombre de _features_ (colonnes) de vos données. Certaines configurations des NN ajoutent un nœud supplémentaire pour un terme de biais.

**Couche de sortie:**

Comme la couche d'entrée, chaque NN possède exactement une couche de sortie. Déterminer sa taille (nombre de neurones) est simple ; elle est entièrement déterminée par la configuration du modèle choisi.

- Si le NN est un régresseur, alors la couche de sortie a un seul nœud (possible plusieurs?).

- Si le NN est un classificateur, alors il a également un seul nœud à moins que la fonction $softmax$ ne soit utilisée, auquel cas la couche de sortie a un nœud par étiquette de classe dans votre modèle.

**Couche cachée:**

Trop peu de neurones peuvent entraîner un sous-ajustement, empêchant le réseau de traiter des données complexes. Trop de neurones peuvent causer un surajustement et rallonger le temps d'entraînement. 

Des règles empiriques suggèrent que le nombre de neurones cachés devrait être défini en fonction de la taille des couches d'entrée et de sortie, mais en fin de compte, la sélection optimale est souvent le résultat d'essais et erreurs.

La règle empirique mentionnée fait référence à plusieurs méthodes couramment utilisées pour estimer le nombre de neurones dans les couches cachées d'un réseau neuronal :

- Le nombre de neurones cachés devrait être compris entre la taille de la couche d'entrée et la taille de la couche de sortie.

- Le nombre de neurones cachés devrait être égal à 2/3 de la taille de la couche d'entrée, plus la taille de la couche de sortie.

- Le nombre de neurones cachés devrait être inférieur au double de la taille de la couche d'entrée.

Ces règles fournissent un point de départ, mais le choix final dépend souvent de tests spécifiques à la tâche et aux données à traiter.

Référence: [Heaton Research - Hidden Layers](heatonresearch.com/2017/06/01/hidden-layers.html)

#### Q5 - Que représentent les vecteurs $\hat{y}$ et $y$? Quelle est la différence entre ces deux quantités?

Les vecteurs $ \hat{y} $ et $ y $ représentent respectivement :

- $ \hat{y} $ : La valeur estimée par le modèle.
- $ y $ : La valeur réelle.

La différence est l'erreur de prédiction du modèle. Et, on cherche à minimiser cette différence.

#### Q6 - Pourquoi utiliser une fonction $softmax$ en sortie ?

Les entrées de la couche $softmax$ sont nommées _logits_. Pour l'évaluation, un $argmax$ sur les logits identifie la classe la plus probable. Pour obtenir une distribution de probabilités, il faut utiliser la fonction $softmax$. Pendant l'entraînement, une fonction de perte basée sur l'entropie croisée mesure la distance entre la distribution de probabilité produite par le réseau et la distribution cible. Cette entropie est définie par $ H = - \sum p_i log(q_i) $ où $p_i$ est la probabilité cible et $q_i$ est la probabilité prédite.


##### Q7 - Ecrire les équations mathématiques permettant d’éffectuer le passe _forward_ du réseau de neurones, c-à-d permettant de produire successivement $\tilde{h}, h, \tilde{y}$ et $\hat{y}$ à partir de $x$.

$$
x : \text{input} \\
\tilde{h} := W_1 x + b_1 : \text{pré-activation de la couche cachée.} \\
h := \sigma(\tilde{h}) : \text{activation de la couche cachée.} \\
\tilde{y} := W_2 h + b_2 : \text{pré-activation de la couche de sortie.} \\
\hat{y} := \sigma(\tilde{y}) : \text{activation de la couche de sortie ( $\hat{y}$ est la prédiction du réseau).}
$$

### 1.3 - Fonction de coût

Grâce à la phase forward, notre réseau de neurones produit pour l'entrée $x^{(i)}$ une sortie $\hat{y}^{(i)}$. On aimerait savoir à quel point cette sortie diffère de la _target_ $y^{(i)}$. Pour cela, on utilise une fonction de coût (loss) adaptée à notre problème. La majorité du temps, la fonction de coût globale $L(X, Y)$ est la moyenne d'une fonction de coût unitaire $\ell(y^{(i)}, \hat{y}^{(i)})$ entre les prédictions et les targets:
$$
L(X, Y) = \frac{1}{N} \sum_{i} \ell(y^{(i)}, f(x^{(i)}))
$$
Il existe de nombreuses fonctions de coût, les deux plus courantes étant :
- l'entropie croisée (cross-entropy) plutôt adaptée aux problèmes de classification :
$$
\ell(y, \hat{y}) = -\sum_{i} y_i \log \hat{y}_i
$$
- l'erreur quadratique (mean squared error, MSE) plutôt adaptée aux problèmes de regression :
$$
\ell(y, \hat{y}) = ||y - \hat{y}||^2_2 = \sum_{i} (y_i - \hat{y}_i)^2
$$

#### Q8 - Pendant l'apprentissage, on cherche à minimiser la fonction de coût. Pour l'entropie croisée et l'erreur quadratique, comment les $\hat{y}_i$ doivent-ils varier pour faire diminuer la _loss_ ?

Pour l'entropie croisée (cross-entropy) :
Lorsque nous traitons des problèmes de classification, le but est que les prédictions $\hat{y}_i$ se rapprochent des étiquettes réelles $y_i$. Dans le cas de la classification binaire, par exemple, si $y_i=1$ (classe positive), alors pour minimiser la perte, $\hat{y}_i$ doit être proche de 1. Inversement, si $y_i=0$, alors $\hat{y}_i$ devrait être proche de 0 pour minimiser l'entropie croisée.

Pour l'erreur quadratique (mean squared error, MSE) :
Le MSE est minimisé lorsque les prédictions $\hat{y}_i$ sont aussi proches que possible des valeurs réelles $y_i$. Cela signifie que pour chaque entrée, la différence entre $\hat{y}_i$ et $y_i$ doit être la plus petite possible. Si $\hat{y}_i$ s'éloigne de $y_i$, le carré de la différence augmente, augmentant ainsi la loss.

#### Q9 - En quoi ces fonctions sont-elles plus adaptées aux problèmes de classification ou de regression ?

Ce n'est pas évident de comprendre pourquoi on devrait préférer l'une à l'autre. Le narratif commun est le suivant:

**Perte d'Entropie Croisée (pour la classification)**:

   - La classification repose souvent sur l'estimation de probabilités. L'entropie croisée mesure la divergence entre les probabilités prédites et les probabilités réelles. Elle pénalise fortement les prédictions qui sont sûres mais incorrectes. En raison de sa nature, elle peut conduire à une convergence plus rapide lors de l'entraînement par rapport à d'autres fonctions de perte.

**Erreur Quadratique Moyenne (pour la régression)**:

   - Dans les problèmes de régression, nous cherchons à prédire une valeur continue. L'erreur quadratique moyenne mesure l'écart entre les valeurs prédites et les valeurs réelles. Elle est directement liée à la variance, une mesure statistique de la dispersion. Minimiser l'EQM revient à maximiser la vraisemblance des prédictions dans le cas où les erreurs suivent une **distribution normale**. Les erreurs plus importantes ont un poids au carré dans l'EQM, ce qui signifie que les prédictions très éloignées de la valeur réelle sont fortement pénalisées, ce qui aide à ajuster le modèle pour éviter de grands écarts.

---

_Distinction théorique:_

L'EQM correspond au modèle de régression paramétrique $Y=f(\theta,X)+\epsilon$, où $\epsilon$ est une erreur aléatoire à moyenne nulle normalement distribuée, indépendante de $X$. Et si vous faites $\epsilon$ non normalement distribué, mais distribué selon une loi de Laplace, vous obtiendrez une autre perte bien connue — l'EMA (erreur moyenne absolue).

La perte d'entropie croisée correspond à un modèle de classification avec deux classes $C_0$ et $C_1$ où $P(C_1|x)=f(\theta,x)$.

Minimiser ces pertes signifie obtenir un estimateur de la vraisemblance maximale pour le paramètre de régression/classification $\theta$.

---

_Invitation à voir plus de détails:_
- Un très beau exemple sur Stats Stackexchange, voir la première réponse - [What is happening here, when I use squared loss in logistic regression setting?](https://stats.stackexchange.com/questions/326350/what-is-happening-here-when-i-use-squared-loss-in-logistic-regression-setting)

- Dans le manuel Deep Learning par Goodfellow, Bengio, et Courville:
> Unfortunately, mean squared error and mean absolute error often lead to poor results when used with gradient-based optimization. Some output units that saturate produce very small gradients when combined with these cost functions. This is one reason that the cross-entropy cost function is more popular than mean squared error or mean absolute error, even when it is not necessary to estimate an entire distribution $p(y|x)$.

### 1.4 - Méthode d'apprentissage

Grâce à la fonction de coût, nous disposons donc d’une mesure de l’erreur de notre réseau de neurones. Nous devons maintenant apprendre ses paramètres (les matrices et vecteurs $W$ et $b$ dans notre exemple) afin de minimiser cette erreur sur l’ensemble des exemples d’apprentissage.

**Descente de gradient** : Pour ce faire, nous allons utiliser l’algorithme de descente de gradient. Il consiste à calculer la dérivée de la fonction de coût par rapport à un paramètre $w$, et à faire un pas dans l’opposé de la direction du gradient, c’est-à-dire à modifier la valeur de $w$ par : 

$$ w \leftarrow w - \eta \frac{\partial L(X, Y)}{\partial w} $$

où $\eta$ est appelé taux d’apprentissage (_learning rate_) et contrôle la vitesse de l’optimisation. Cette étape est répétée un nombre prédéfini d’itérations ou jusqu’à convergence de l’apprentissage.

Le gradient $\frac{\partial L(X, Y)}{\partial w}$ peut être calculé sur différents ensembles, ce qui correspond à différentes variantes de l’algorithme de descente de gradient:

- l’ensemble d’apprentissage (_train_) complet, c’est la descente de gradient classique;
- un petit sous-ensemble (généralement quelques dizaines) d’exemples d’apprentissage tirés aléatoirement, c’est la descente de gradient stochastique par mini-batch (stochastic gradient descent, SGD);
- une seule paire d’exemple $(x(i), y(i))$, on remplace donc $L$ par le coût unitaire $\ell$, c’est la descente de gradient stochastique online.

**Backpropagation** : Pour calculer le gradient de la sortie par rapport aux poids du réseau, on applique le théorème de dérivation des fonctions composées (_chain rule_) :

$$
\frac{\partial c}{\partial a} = \frac{\partial c}{\partial b} \frac{\partial b}{\partial a}
$$

Dans le cas vectoriel, si $a, b, c$ sont des vecteurs, on a pour la dérivée de la composante $c_i$ par rapport à la composante $a_k$:

$$
\frac{\partial c_i}{\partial a_k} = \sum_j \frac{\partial c_i}{\partial b_j} \frac{\partial b_j}{\partial a_k} \quad (2)
$$

Appliqué de façon naïve, le calcul des gradients applique à nouveau la _chain rule_ depuis la _loss_ pour chaque couche. Cela est très inefficace et il est possible de faire mieux en utilisant le principe de la backpropagation du gradient, qui consiste à chainer les gradients pour ne parcourir qu’une seule fois le réseau. 

On visualise schématiquement l’application de la backpropagation sur un réseau de neurones sur la [figure 2](../resources/nn_figure_2.PNG). Le principe est que le gradient calculé par rapport à la sortie d’une couche peut être réutilisé pour calculer les gradients par rapport à l’entrée et aux paramètres de cette même couche : il suffit de le multiplier par un simple gradient local (de la sortie par rapport à l’entrée ou aux paramètres). 

Le calcul successif des gradients des différentes couches s’appelle la phase _backward_ du réseau.

#### Q10 - Quels semblent être les avantages et inconvénients des diverses variantes de descente de gradient entre les versions classique, stochastique sur mini-batch et stochastique online? Laquelle semble la plus raisonnable à utiliser dans le cas général ?

**Formules sous forme de tableau** :

| Méthode | Formule de mise à jour |
|---------|------------------------|
| Classique | $ w = w - \eta \nabla J(w) $ où $ \nabla J(w) $ est le gradient calculé sur l'ensemble de données |
| SGD | $ w = w - \eta \nabla j(w; x^{(i)}, y^{(i)}) $ où $ (x^{(i)}, y^{(i)}) $ est un exemple unique |
| Mini-batch | $ w = w - \eta \nabla j(w; X_{\text{mini-batch}}, Y_{\text{mini-batch}}) $ où $ X_{\text{mini-batch}}, Y_{\text{mini-batch}} $ est un sous-ensemble aléatoire du dataset |

Ici, $ \eta $ est le taux d'apprentissage, $ w $ est le vecteur de poids, $ J(w) $ est la fonction de coût sur l'ensemble de données, et $ j(w; x, y) $ est la fonction de coût pour un exemple individuel.

**Avantages**:
| **Stochastic Learning**                    | **Batch Learning**                 |
|-----------------------------------------------------------------|-------------------------------------------------------------------------|
| Habituellement beaucoup plus rapide que l'apprentissage par lots | Les conditions de convergence sont bien comprises                      |
| Aboutit à de meilleures solutions                               | De nombreuses techniques d'accélération (ex. gradient conjugué) fonctionnent uniquement en apprentissage par lots |
| Peut être utilisé pour suivre les changements                    | L'analyse théorique de la dynamique des poids et des taux de convergence est plus simple         |

<h4 style="color:orange">...A ajouter plus de détails </h4>

#### Q11 - Quelle est l’influence du learning rate $ \eta $ sur l’apprentissage ?
   
   Le taux d'apprentissage, souvent désigné par $ \eta $, détermine la taille des pas que l'algorithme de descente de gradient fait pour atteindre un minimum local/global. 
   * Un $ \eta $ trop élevé peut causer des oscillations autour du minimum ou même diverger.
   * Un $ \eta $ trop faible peut rendre la convergence très lente.
   * Le choix optimal de $ \eta $ dépend souvent du problème et peut nécessiter des essais et des erreurs ou des techniques adaptatives pour le définir.

#### Q12 - Comparer la complexité (en fonction du nombre de couches du réseau) du calcul des gradients de la _loss_ par rapport aux paramètres, en utilisant l’approche naïve et l’algorithme de backprop.
   
Pour chaque couche, nous devons appliquer la règle de la chaîne à chaque fois, ce qui nécessitera une multiplication de matrices pour chaque couche. La complexité serait $O(n^2)$ pour chaque couche, donc pour $ L $ couches, la complexité serait $ O(n^2L) $.

Avec la backpropagation, les gradients sont propagés efficacement à travers le réseau, ce qui réduit les calculs redondants. La complexité serait essentiellement $ O(n^2) $ pour un réseau avec $ L $ couches.

#### Q13 - Quel critère doit respecter l’architecture du réseau pour permettre la backpropagation ?
   
   L'architecture du réseau doit utiliser des fonctions d'activation et des fonctions de perte différentiables pour permettre la backpropagation.

##### Q14 - La fonction $softmax$ et la _loss_ de $crossentropy$ sont souvent utilisées ensemble et leur gradient est très simple. Montrez que la _loss_ se simplifie en  $l = -\sum_{i} y_i \tilde{y}_i + \log\left(\sum_{j=1}^{K} e^{\tilde{y}_j}\right) $

Le $softmax$ est défini comme suivant pour un vecteur $ \tilde{y} $ de dimension $ K $:
$ softmax(\tilde{y})_i = \frac{e^{\tilde{y}_i}}{\sum_{j=1}^{K} e^{\tilde{y}_j}} $

où $ i $ est un indice spécifique dans ce vecteur.

La $crossentropy$ entre les véritables étiquettes $ y $ et les prédictions $ \hat{y} $ est :
$ L(y, \hat{y}) = -\sum_{i} y_i \log(\hat{y}_i) $

où $ \hat{y}_i $ est la prédiction $i-ème$ obtenue après avoir appliqué $softmax$ à $ \tilde{y} $.

Insérons notre expression $softmax$ pour $ \hat{y}_i $ dans la fonction de coût :
$$ L(y, \tilde{y}) = -\sum_{i} y_i \log\left(\frac{e^{\tilde{y}_i}}{\sum_{j=1}^{K} e^{\tilde{y}_j}}\right) \Rightarrow L(y, \tilde{y}) = -\sum_{i} y_i \tilde{y}_i + \sum_{i} y_i \log\left(\sum_{j=1}^{K} e^{\tilde{y}_j}\right) $$

Puisque $ \sum_{i} y_i = 1 $, nous obtenons :
$$ L(y, \tilde{y}) = -\sum_{i} y_i \tilde{y}_i + \log\left(\sum_{j=1}^{K} e^{\tilde{y}_j}\right) $$